This is a Jupyter notebook that demonstrates how various sorts of rebreathers
work in some relatively coherent fashion. My aim was to make this project such that even without a detailed understanding of what these machines do, the differences in them should be readily apparent.

Here is a crash course in rebreathers:

A rebreather is a machine used for SCUBA diving. When we dive, we generally exhale directly into the water. As we go deeper, we use more air. A rebreather is a machine that adds oxygen into the mix, and removes the carbon dioxide we exhale. They are complex and unreliable machines, and technical divers devote a great deal of time and attention to ensuring that they do not kill us.

When I first began diving rebreathers, there was a popular sticker and it said the following:

THIS MACHINE HAS NO BRAIN. USE YOUR OWN.

It adorned my hacked-up, garbage-can rebreather for many years. I would highly advise against building one's own rebreather, and in general, I'd advise against building your own life support equipment for any scenario. Nowadays, many rebreathers do have brains, so to speak, but this has certainly not made them any more reliable. The exact failure modes and mechanisms of a rebreather are (far) beyond the scope of this work, but virtually all of them are deadly.

Fundamentally, you are a machine that metabolizes oxygen, and you regurigtate carbon dioxide in varying amounts, usually depending on what you're doing and what you've eaten. When you dive, you take up inert gas - in this example, nitrogen, but in theory you can do it with any inert gas. The only other important one is helium, whose dynamic properties are too complex to discuss here.

More background is required: You cannot breathe pure oxygen below about 20 feet. It causes severe seizures, severe enough to cause permanent spinal trauma - but they're usually fatal before that point. We have to breathe some inert gas, which we call "diluent", since it dilutes the oxygen for us. With a rebreather, you can breathe as much oxygen as is safe for a given depth, meaning you breathe less inert gas - this means considerably less of it actually absorbs into your body. Or, in so many words, you incur a significantly shortened decompression penalty when you use a rebreather. How much shorter depends on what type of rebreather you use.

Broadly speaking, there are two different sorts of rebreathers: Semi-closed circuit and closed-circuit. Mathematically speaking, all rebreathers are actually semi-closed, it's just that closed circuit rebreathers do not dump off any gas during normal operation. A semi-closed circuit rebreather (sometimes called a "gas extender") dumps a certain fraction of the breathed gas out, replacing it with a gas mixture, not pure oxygen. You only really absorb about 20% of the oxygen in any given breath, so a given volume of gas is usually good for about 5 go-rounds as long as CO2 is removed. Closed circuit rebreathers do, however, directly add pure oxygen back into the mixture.

The code below allows you to choose a dive profile, and then see what your tissue loading (analogous to decompression penalty) would be for each rebreather type, comparing two given types of diving.

Below, in a separate cell, I've included all relevant equations with as much explanation as I felt was bearable.

This code was significantly updated so that mCCR and SCR behave the same way, as they should. However, remember, rebreathers are already quite deadly, and sketchy code written by someone on the internet only exacerbates matters. This is only to help you understand why a given system incurs the decompression penalty that it does, it cannot be used as a dive planner.

As an aside, if the oceans aren't clean, there's nothing to look at when you dive. Please try to keep the oceans clean. We aren't going to get new ones any time soon.



### 1. Ambient Pressure

$$P_{\mathrm{amb}}(\text{depth}) \;=\; 1.0 \;+\; \frac{\text{depth in meters}}{10} \quad\text{(bar)}$$

**Meaning:** At the surface, $P_{\mathrm{amb}} \approx 1.0\,\mathrm{bar}$, and each $10\,\mathrm{m}$ of depth adds about $1\,\mathrm{bar}$.

### 2. Oxygen Inflow (Constant Mass Flow)

$$\dot{V}_{\mathrm{O_2,in}} \;=\; \dot{V}_{\mathrm{cmf}} \;\times\; F_{\mathrm{O_2,\,supply}}$$

**Meaning:** If the orifice flow $\dot{V}_{\mathrm{cmf}}$ is in $\mathrm{L\,min}^{-1}$ and the supply gas has an oxygen fraction  $F_{\mathrm{O_2,\,supply}}$, the net O$_2$ inflow is the product of these.

### 3. Oxygen Outflow (Metabolism and Venting)

$$\dot{V}_{\mathrm{O_2,out}} \;=\; \dot{V}_{\mathrm{O_2,\,consumed}} \;+\; \bigl(\dot{V}_{\mathrm{vent}} \times F_{\mathrm{O_2}}(\text{loop})\bigr)$$

**Meaning:** In a semi‐closed system, we lose oxygen via (i) metabolic consumption $\dot{V}_{\mathrm{O_2,\,consumed}}$ and (ii) forced vent flow $\dot{V}_{\mathrm{vent}}$, multiplied by the loop fraction of O$_2$.

### 4. Metabolic Oxygen Consumption

$$\dot{V}_{\mathrm{O_2,\,consumed}} \;=\; \text{constant (L/min)}$$

**Meaning:** A typical simplified model uses a constant oxygen consumption rate based on the diver’s workload (e.g. $0.8\,\mathrm{L/min}$).

### 5. Net Rate of Change in Loop  $F_{\mathrm{O_2}}$

$$\frac{d\,F_{\mathrm{O_2}}}{dt} \;=\; \frac{ \dot{V}_{\mathrm{O_2,in}} \;-\; \dot{V}_{\mathrm{O_2,out}} }{ V_{\mathrm{loop}} } \quad.$$

**Meaning:** The time derivative of loop oxygen fraction is the net O$_2$ flow rate in minus out, divided by the loop volume $V_{\mathrm{loop}}$ (in liters).

### 6. Euler Update for a Small Time Step $\Delta t$

$$F_{\mathrm{O_2}}^{(\mathrm{new})} \;=\; F_{\mathrm{O_2}}^{(\mathrm{old})} \;+\; \left[  \frac{\dot{V}_{\mathrm{O_2,in}} - \dot{V}_{\mathrm{O_2,out}}}{V_{\mathrm{loop}}} \right] \,\Delta t$$

$$\mathrm{ppO_2}^{(\mathrm{new})} \;=\; P_{\mathrm{amb}} \;\times\; F_{\mathrm{O_2}}^{(\mathrm{new})}$$

**Meaning:** Over a small interval $\Delta t$, we do a simple (Euler) increment to update loop fraction. Then partial pressure of O$_2$ is fraction times the ambient pressure.

For context, here are diagrams of all the various sorts of rebreathers you can use here:

Here's the diagram for both mCCR and eCCR:

![link text](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Mixed_gas_CCR_loop_schematic.png/659px-Mixed_gas_CCR_loop_schematic.png)

Peter Southwood, [Mixed gas CCR loop schematic](https://commons.wikimedia.org/wiki/File:Mixed_gas_CCR_loop_schematic.png), [CC license](https://creativecommons.org/licenses/by-sa/3.0/legalcode)

1 Dive/surface valve and loop non-return valves

2 Exhaust hose

3 Scrubber (axial flow)

4 Counterlung

5 Overpressure valve

6 Inhalation valve

7 Oxygen cylinder

8 Oxygen cylinder valve

9 Absolute pressure oxygen regulator

10 Oxygen submersible pressure gauge

11 Oxygen manual bypass valve

12 Oxygen constant mass flow metering orifice

13 Electronically controlled solenoid operated oxygen injection valve

14 Diluent cylinder

15 Diluent cylinder valve

16 Diluent regulator

17 Diluent submersible pressure gauge

18 Bailout demand valve

19 Manual diluent bypass valve

20 Automatic diluent valve

21 Electronic control and monitoring circuits

22 Primary and secondary display units

This image is actually cheating a bit - this demonstrates a "hybrid" CCR with both an electronic solenoid and a constant mass flow, so it's both eCCR and mCCR. For eCCR alone, omit 12, and for mCCR alone, omit 13. I know, clear as mud.

For SCR, it's like so:
![link text](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/CMF_SCR_loop_schematic.png/800px-CMF_SCR_loop_schematic.png)

Peter Southwood [CMF CR loop schematic](https://commons.wikimedia.org/wiki/File:CMF_SCR_loop_schematic.png), [CC License](https://creativecommons.org/licenses/by-sa/3.0/legalcode)




In [45]:

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Markdown

# 16 BÜHLMANN N2 half-times (minutes)
buhlmann_N2_half_times = [
    4.0, 8.0, 12.5, 18.5, 27.0, 38.3, 54.3, 77.0,
    109.0, 146.0, 187.0, 239.0, 305.0, 390.0, 498.0, 635.0
]

def update_loop_fraction(
    rebreather_type,
    current_FO2,
    depth_m,
    dt_min,
    loop_volume,
    metabolic_O2,         # L/min
    # For SCR:
    scr_cmf_flow=4.1,     # L/min air
    scr_vent_flow=0.3,
    scr_supply_frac=0.21,
    # For mCCR:
    mccr_cmf_flow=1.2,    # L/min O2 (Adjusted default - INCREASED)
    mccr_supply_frac=1.0,
    # eCCR:
    eccr_setpoint=1.3
):
    """
    Single-step update of O2 fraction in a 'loop'.

    Args:
        rebreather_type (str): "OC", "SCR", "mCCR", or "eCCR".
        current_FO2 (float): Current fraction of O2 in the loop.
        depth_m (float): Depth in meters.
        dt_min (float): Time step in minutes.
        loop_volume (float): Volume of the breathing loop in liters.
        metabolic_O2 (float): Metabolic oxygen consumption rate in L/min.
        scr_cmf_flow (float): SCR constant mass flow rate in L/min.
        scr_vent_flow (float): SCR vent flow rate in L/min.
        scr_supply_frac (float): SCR supply gas O2 fraction.
        mccr_cmf_flow (float): mCCR constant mass flow rate in L/min.
        mccr_supply_frac (float): mCCR supply gas O2 fraction.
        eccr_setpoint (float): eCCR setpoint in bar.

    Returns:
        tuple: (new_FO2, new_ppO2) - new O2 fraction and ppO2.
    """
    # Ambient pressure in bar
    P_amb = 1.0 + depth_m/10.0
    current_ppO2 = P_amb * current_FO2

    # OPEN CIRCUIT
    if rebreather_type == "OC":
        new_FO2 = scr_supply_frac  # typically 0.21 if "air"
        new_ppO2 = P_amb * new_FO2
        return new_FO2, new_ppO2

    # eCCR
    if rebreather_type == "eCCR":
        desired_fraction = eccr_setpoint / P_amb
        desired_fraction = max(0.0, min(1.0, desired_fraction))
        new_ppO2 = P_amb * desired_fraction
        return desired_fraction, new_ppO2

    # SCR or mCCR
    if rebreather_type == "SCR":
        o2_in = scr_cmf_flow * scr_supply_frac
        o2_out = (scr_vent_flow * current_FO2) + metabolic_O2
    elif rebreather_type == "mCCR":
        o2_in = mccr_cmf_flow * mccr_supply_frac # Constant flow of pure O2
        o2_out = metabolic_O2  # no forced vent
    else:
        raise ValueError(f"Invalid rebreather_type: {rebreather_type}")


    dFO2_dt = (o2_in - o2_out) / loop_volume
    new_FO2 = current_FO2 + dFO2_dt * dt_min
    new_FO2 = max(0.0, min(1.0, new_FO2))
    new_ppO2 = (1.0 + depth_m/10.0)*new_FO2
    return new_FO2, new_ppO2

def update_tissues(tissue_pressures, dt_min, inert_fraction, ambient_pressure, half_times):
    """
    Incremental update for inert partial pressure (N2) in 16 compartments.

    Args:
        tissue_pressures (list): List of current tissue pressures.
        dt_min (float): Time step in minutes.
        inert_fraction (float): Fraction of inert gas in the breathing mix.
        ambient_pressure (float): Ambient pressure in bar.
        half_times (list): List of N2 half-times for the compartments.

    Returns:
        list: Updated tissue pressures.
    """
    new_ps = []
    pinert = ambient_pressure * inert_fraction
    for p_old, t_half in zip(tissue_pressures, half_times):
        k = np.log(2)/t_half
        p_new = p_old*np.exp(-k*dt_min) + pinert*(1-np.exp(-k*dt_min))
        new_ps.append(p_new)
    return new_ps

def simulate_square_dive(
    rebreather_type="SCR",
    bottom_depth=30.0,
    bottom_time=20.0,
    total_time=40.0,
    dt=0.25,
    loop_volume=6.0,
    metabolic_O2=0.8,
    # For SCR:
    scr_cmf_flow=4.1,
    scr_vent_flow=0.3,
    scr_supply_frac=0.21,
    # For mCCR:
    mccr_cmf_flow=1.2, # Adjusted default - INCREASED
    mccr_supply_frac=1.0,
    # eCCR:
    eccr_setpoint=1.3
):
    """
    Square dive simulation.

    Args:
        rebreather_type (str): "OC", "SCR", "mCCR", or "eCCR".
        bottom_depth (float): Maximum depth of the dive in meters.
        bottom_time (float): Time spent at the bottom depth in minutes.
        total_time (float): Total dive time in minutes.
        dt (float): Time step for the simulation in minutes.
        loop_volume (float): Volume of the breathing loop in liters.
        metabolic_O2 (float): Metabolic oxygen consumption in L/min.
        scr_cmf_flow (float): SCR constant mass flow rate in L/min.
        scr_vent_flow (float): SCR vent flow rate in L/min.
        scr_supply_frac (float): SCR supply gas O2 fraction.
        mccr_cmf_flow (float): mCCR constant mass flow rate in L/min.
        mccr_supply_frac (float): mCCR supply gas O2 fraction (usually 1.0).
        eccr_setpoint (float): eCCR setpoint in bar.

    Returns:
        tuple: (times, ppo2_list, tiss_history) - simulation time, ppO2, and tissue N2.

    """
    if total_time < bottom_time+4:
        total_time = bottom_time+4

    # Build a profile
    profile = [
        (0.0, 0.0),
        (2.0, bottom_depth),
        (2.0+bottom_time, bottom_depth),
        (4.0+bottom_time, 0.0),
        (total_time, 0.0)
    ]
    times = np.arange(0, total_time+dt, dt)
    tissues = [0.79]*16

    # initial fraction
    if rebreather_type=="OC":
        FO2 = scr_supply_frac
    else:
        FO2 = 0.21

    ppo2_list=[]
    tiss_history=[]

    def get_depth(tmin):
        """Calculates depth at a given time."""
        for i in range(len(profile)-1):
            t0,d0 = profile[i]
            t1,d1 = profile[i+1]
            if tmin>=t0 and tmin<t1:
                alpha=(tmin-t0)/(t1-t0+1e-9)
                return d0+alpha*(d1-d0)
        return profile[-1][1]

    for tm in times:
        depth_m = get_depth(tm)
        new_FO2, new_ppO2 = update_loop_fraction(
            rebreather_type=rebreather_type,
            current_FO2=FO2,
            depth_m=depth_m,
            dt_min=dt,
            loop_volume=loop_volume,
            metabolic_O2=metabolic_O2,
            scr_cmf_flow=scr_cmf_flow,
            scr_vent_flow=scr_vent_flow,
            scr_supply_frac=scr_supply_frac,
            mccr_cmf_flow=mccr_cmf_flow,
            mccr_supply_frac=mccr_supply_frac,
            eccr_setpoint=eccr_setpoint
        )

        P_amb = 1.0 + depth_m/10.0
        inert_fraction = 1.0 - new_FO2
        tissues = update_tissues(tissues, dt, inert_fraction, P_amb, buhlmann_N2_half_times)

        FO2=new_FO2
        ppo2_list.append(new_ppO2)
        tiss_history.append(tissues[:])

    return times, np.array(ppo2_list), np.array(tiss_history)


# ------------------ WIDGETS GUI ------------------
# Dive Profile Dropdown
dive_profiles = {
    "Challenging 40m/30min": {"depth": 40.0, "bottom_time": 30.0, "total_time": 70.0},
    "Pushing Limits 50m/45min":         {"depth": 50.0, "bottom_time": 45.0, "total_time": 100.0},
    "Extended Shallow 20m/60min":      {"depth": 20.0, "bottom_time": 60.0, "total_time": 90.0},
    "Custom":                 None # Use sliders
}
dive_profile_widget = widgets.Dropdown(
    options=list(dive_profiles.keys()),
    value="Challenging 40m/30min",
    description="Dive Profile"
)

# Rebreather Type Dropdowns
rebreather_type_widget_1 = widgets.Dropdown(
    options=["OC", "SCR", "mCCR", "eCCR"],
    value="OC",
    description="Type 1"
)
rebreather_type_widget_2 = widgets.Dropdown(
    options=["OC", "SCR", "mCCR", "eCCR"],
    value="eCCR",
    description="Type 2"
)

# Sliders - Depth, Time, dt
bottom_depth_widget = widgets.FloatSlider(value=30, min=0, max=100, step=1, description="Depth(m)")
bottom_time_widget = widgets.FloatSlider(value=20, min=5, max=120, step=5, description="BtmTime")
total_time_widget = widgets.FloatSlider(value=40, min=10, max=240, step=5, description="TotTime")
dt_widget = widgets.FloatSlider(value=0.25, min=0.05, max=1.0, step=0.05, description="dt(min)")

# Loop Volume and MetabO2 - Conditional Visibility - COMMON to both
loop_vol_widget = widgets.FloatSlider(value=6.0, min=3.0, max=15.0, step=0.5, description="LoopVol")
metabO2_widget = widgets.FloatSlider(value=0.8, min=0.1, max=2.0, step=0.1, description="O2Cons")
loop_params_ui = widgets.VBox([
    loop_vol_widget,
    metabO2_widget
])


# SCR Parameters - COLUMN 1
scr_cmf_flow_widget_1 = widgets.FloatSlider(value=4.1, min=0.0, max=8.0, step=0.1, description="SCR CMF")
scr_vent_widget_1 = widgets.FloatSlider(value=0.3, min=0.0, max=2.0, step=0.1, description="SCR Vent")
scr_supply_frac_widget_1 = widgets.FloatSlider(value=0.21, min=0.21, max=1.0, step=0.01, description="SCR F_O2")
scr_params_ui_1 = widgets.VBox([
    scr_cmf_flow_widget_1,
    scr_vent_widget_1,
    scr_supply_frac_widget_1
])

# mCCR Parameters - COLUMN 1
mccr_cmf_widget_1 = widgets.FloatSlider(value=1.2, min=0.0, max=5.0, step=0.1, description="mCCR O2 CMF") # Adjusted default - INCREASED
mccr_supply_frac_widget_1 = widgets.FloatSlider(value=1.0, min=0.21, max=1.0, step=0.01, description="mCCR F_O2")
mccr_params_ui_1 = widgets.VBox([
    mccr_cmf_widget_1,
    mccr_supply_frac_widget_1
])

# eCCR Parameters - COLUMN 1
eccr_setpt_widget_1 = widgets.FloatSlider(value=1.3, min=0.5, max=1.6, step=0.05, description="eCCR Setpt")
eccr_params_ui_1 = widgets.VBox([
    eccr_setpt_widget_1
])


# SCR Parameters - COLUMN 2
scr_cmf_flow_widget_2 = widgets.FloatSlider(value=4.1, min=0.0, max=8.0, step=0.1, description="SCR CMF")
scr_vent_widget_2 = widgets.FloatSlider(value=0.3, min=0.0, max=2.0, step=0.1, description="SCR Vent")
scr_supply_frac_widget_2 = widgets.FloatSlider(value=0.21, min=0.21, max=1.0, step=0.01, description="SCR F_O2")
scr_params_ui_2 = widgets.VBox([
    scr_cmf_flow_widget_2,
    scr_vent_widget_2,
    scr_supply_frac_widget_2
])

# mCCR Parameters - COLUMN 2
mccr_cmf_widget_2 = widgets.FloatSlider(value=1.2, min=0.0, max=5.0, step=0.1, description="mCCR O2 CMF") # Adjusted default - INCREASED
mccr_supply_frac_widget_2 = widgets.FloatSlider(value=1.0, min=0.21, max=1.0, step=0.01, description="mCCR F_O2")
mccr_params_ui_2 = widgets.VBox([
    mccr_cmf_widget_2,
    mccr_supply_frac_widget_2
])

# eCCR Parameters - COLUMN 2
eccr_setpt_widget_2 = widgets.FloatSlider(value=1.3, min=0.5, max=1.6, step=0.05, description="eCCR Setpt")
eccr_params_ui_2 = widgets.VBox([
    eccr_setpt_widget_2
])


# Initial visibility setup - all parameter boxes hidden
loop_params_ui.layout.visibility = 'hidden'
scr_params_ui_1.layout.visibility = 'hidden'
mccr_params_ui_1.layout.visibility = 'hidden'
eccr_params_ui_1.layout.visibility = 'hidden'
scr_params_ui_2.layout.visibility = 'hidden'
mccr_params_ui_2.layout.visibility = 'hidden'
eccr_params_ui_2.layout.visibility = 'hidden'


def update_param_visibility_1(change):
    """COLUMN 1 visibility."""
    if change['new'] == 'SCR':
        scr_params_ui_1.layout.visibility = 'visible'
    else:
        scr_params_ui_1.layout.visibility = 'hidden'

    if change['new'] == 'mCCR':
        mccr_params_ui_1.layout.visibility = 'visible'
    else:
        mccr_params_ui_1.layout.visibility = 'hidden'

    if change['new'] == 'eCCR':
        eccr_params_ui_1.layout.visibility = 'visible'
    else:
        eccr_params_ui_1.layout.visibility = 'hidden'


def update_param_visibility_2(change):
    """COLUMN 2 visibility."""
    if change['new'] == 'SCR':
        scr_params_ui_2.layout.visibility = 'visible'
    else:
        scr_params_ui_2.layout.visibility = 'hidden'

    if change['new'] == 'mCCR':
        mccr_params_ui_2.layout.visibility = 'visible'
    else:
        mccr_params_ui_2.layout.visibility = 'hidden'

    if change['new'] == 'eCCR':
        eccr_params_ui_2.layout.visibility = 'visible'
    else:
        eccr_params_ui_2.layout.visibility = 'hidden'


def update_loop_params_visibility(change):
    """Updates visibility of loop_params_ui."""
    type1 = rebreather_type_widget_1.value
    type2 = rebreather_type_widget_2.value
    if type1 in ["SCR", "mCCR", "eCCR"] or type2 in ["SCR", "mCCR", "eCCR"]:
        loop_params_ui.layout.visibility = 'visible'
    else:
        loop_params_ui.layout.visibility = 'hidden'

# Observe changes in rebreather types, SEPARATELY for each column
rebreather_type_widget_1.observe(update_param_visibility_1, names='value')
rebreather_type_widget_2.observe(update_param_visibility_2, names='value')

# Also, make sure loop params visibility updates correctly on *either* change
rebreather_type_widget_1.observe(update_loop_params_visibility, names='value')
rebreather_type_widget_2.observe(update_loop_params_visibility, names='value')


def link_mccr_cmf_to_metabo2(change):
    """Link mCCR CMF to Metabolic O2 (initial value)."""
    # Only set if mCCR is selected.  Important for correct behavior.
    if rebreather_type_widget_1.value == 'mCCR':
        mccr_cmf_widget_1.value = change.new
    if rebreather_type_widget_2.value == 'mCCR':
        mccr_cmf_widget_2.value = change.new

metabO2_widget.observe(link_mccr_cmf_to_metabo2, names='value')



def interactive_plot(
    dive_profile_name,
    rebreather_type_1, rebreather_type_2,
    bottom_depth, bottom_time, total_time, dt,
    loop_volume, metabO2,
    scr_cmf_flow_1, scr_vent_1, scr_supply_frac_1,
    mccr_cmf_1, mccr_supply_frac_1,
    eccr_setpt_1,
    scr_cmf_flow_2, scr_vent_2, scr_supply_frac_2,
    mccr_cmf_2, mccr_supply_frac_2,
    eccr_setpt_2
):
    """
    Generates interactive plots comparing two rebreather configurations.

    Args: (Same as the individual parameter widgets, plus dive_profile_name)
        dive_profile_name (str):  Name of the selected dive profile.

    Returns:
        None (displays plots and Markdown output).
    """

    if dive_profile_name != "Custom":
        profile = dive_profiles[dive_profile_name]
        bottom_depth = profile["depth"]
        bottom_time = profile["bottom_time"]
        total_time = profile["total_time"]

    # --- Simulation 1 ---
    t1, ppo2_arr_1, tiss_2d_1 = simulate_square_dive(
        rebreather_type=rebreather_type_1,
        bottom_depth=bottom_depth,
        bottom_time=bottom_time,
        total_time=total_time,
        dt=dt,
        loop_volume=loop_volume,
        metabolic_O2=metabO2,
        scr_cmf_flow=scr_cmf_flow_1,
        scr_vent_flow=scr_vent_1,
        scr_supply_frac=scr_supply_frac_1,
        mccr_cmf_flow=mccr_cmf_1,
        mccr_supply_frac=mccr_supply_frac_1,
        eccr_setpoint=eccr_setpt_1
    )

    # --- Simulation 2 ---
    t2, ppo2_arr_2, tiss_2d_2 = simulate_square_dive(
        rebreather_type=rebreather_type_2,
        bottom_depth=bottom_depth,
        bottom_time=bottom_time,
        total_time=total_time,
        dt=dt,
        loop_volume=loop_volume,
        metabolic_O2=metabO2,
        scr_cmf_flow=scr_cmf_flow_2,
        scr_vent_flow=scr_vent_2,
        scr_supply_frac=scr_supply_frac_2,
        mccr_cmf_flow=mccr_cmf_2,
        mccr_supply_frac=mccr_supply_frac_2,
        eccr_setpoint=eccr_setpt_2
    )


    fig, axes = plt.subplots(1,2, figsize=(12,5))

    max_ppo2 = max(np.max(ppo2_arr_1), np.max(ppo2_arr_2))
    max_pn2 = max(np.max(tiss_2d_1), np.max(tiss_2d_2))

    axes[0].plot(t1, ppo2_arr_1, label=f"{rebreather_type_1} Loop ppO2", color="blue")
    axes[0].plot(t2, ppo2_arr_2, label=f"{rebreather_type_2} Loop ppO2", color="red")
    axes[0].set_xlabel("Time (min)")
    axes[0].set_ylabel("ppO2 (bar)")
    axes[0].set_title("Loop ppO2 Comparison")
    axes[0].grid(True)
    axes[0].legend()
    axes[0].set_ylim(0, max_ppo2 * 1.1)

    for i in range(tiss_2d_1.shape[1]):
        axes[1].plot(t1, tiss_2d_1[:,i], color="blue", alpha=0.5)
        axes[1].plot(t2, tiss_2d_2[:,i], color="red", alpha=0.5)
    axes[1].set_xlabel("Time (min)")
    axes[1].set_ylabel("PN2 (bar)")
    axes[1].set_title("16-Compartment N2 Comparison")
    axes[1].grid(True)
    axes[1].set_ylim(0, max_pn2 * 1.1)

    plt.tight_layout()
    plt.show()


    max_tiss_n2_1 = np.max(tiss_2d_1[-1,:])
    max_tiss_n2_2 = np.max(tiss_2d_2[-1,:])

    if max_tiss_n2_1 < max_tiss_n2_2:
        decomp_text = f"{rebreather_type_1} is predicted to have a lower decompression obligation than {rebreather_type_2} for this dive profile (based on maximum tissue N2)."
    elif max_tiss_n2_2 < max_tiss_n2_1:
        decomp_text = f"{rebreather_type_2} is predicted to have a lower decompression obligation than {rebreather_type_1} for this dive profile (based on maximum tissue N2)."
    else:
        decomp_text = "Both rebreather types are predicted to have a similar decompression obligation for this dive profile (based on maximum tissue N2)."

    display(Markdown(f"**Decompression Comparison:** {decomp_text}  \n**Important:** This is a simplified model for demonstration purposes only and should not be used for actual dive planning."))



# --- UI Layout ---

rebreather_1_params_ui = widgets.VBox([
    rebreather_type_widget_1,
    scr_params_ui_1,
    mccr_params_ui_1,
    eccr_params_ui_1
])

rebreather_2_params_ui = widgets.VBox([
    rebreather_type_widget_2,
    scr_params_ui_2,
    mccr_params_ui_2,
    eccr_params_ui_2
])


ui = widgets.VBox([
    widgets.HBox([dive_profile_widget]),
    widgets.HBox([rebreather_1_params_ui, rebreather_2_params_ui, dt_widget]),
    widgets.HBox([loop_params_ui]),
    widgets.HBox([bottom_depth_widget, bottom_time_widget, total_time_widget]),
])


out = widgets.interactive_output(
    interactive_plot,
    {
        "dive_profile_name": dive_profile_widget,
        "rebreather_type_1": rebreather_type_widget_1,
        "rebreather_type_2": rebreather_type_widget_2,
        "bottom_depth": bottom_depth_widget,
        "bottom_time": bottom_time_widget,
        "total_time": total_time_widget,
        "dt": dt_widget,
        "loop_volume": loop_vol_widget,
        "metabO2": metabO2_widget,
        "scr_cmf_flow_1": scr_cmf_flow_widget_1,
        "scr_vent_1": scr_vent_widget_1,
        "scr_supply_frac_1": scr_supply_frac_widget_1,
        "mccr_cmf_1": mccr_cmf_widget_1,
        "mccr_supply_frac_1": mccr_supply_frac_widget_1,
        "eccr_setpt_1": eccr_setpt_widget_1,
        "scr_cmf_flow_2": scr_cmf_flow_widget_2,
        "scr_vent_2": scr_vent_widget_2,
        "scr_supply_frac_2": scr_supply_frac_widget_2,
        "mccr_cmf_2": mccr_cmf_widget_2,
        "mccr_supply_frac_2": mccr_supply_frac_widget_2,
        "eccr_setpt_2": eccr_setpt_widget_2
    }
)

display(ui, out)

Output()